In [1]:
import requests
from bs4 import BeautifulSoup
import pickle
import re
import json

In [3]:
DOMAIN = "http://www.filmcritics.org.hk"

headers = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36"
}

resp = requests.get(DOMAIN + "/taxonomy/term/1?page=0", headers = headers)
resp.text

movie_dict_list = []

pageurl_list = []

lastpg = int(re.findall('href="(.+)" title="到最後一頁"', resp.text)[0].split('page=')[1])  #給出最後一頁的頁數,找出總共有幾頁
for pg in range(0, lastpg):
    p = DOMAIN + "/taxonomy/term/1?page=%s"%pg
    pageurl_list.append(p)
# print(page_list) 

with open('C:\\Users\\Java\\ETL\\movie\\hkfcsUrls','wb') as f:
    pickle.dump(pageurl_list,f)

In [5]:
with open('C:\\Users\\Java\\ETL\\movie\\hkfcsUrls','rb') as f:
    page_list = pickle.load(f)
    
with open('C:\\Users\\Java\\ETL\\movie\\InJ_compare','rb') as f:
    InJ_compare = pickle.load(f)


article_urls = []
movie_dict = {}
# counter = 0

for page in page_list:
    resp = requests.get(page, headers = headers)
    soup = BeautifulSoup(resp.text, 'lxml')
    for i in range(1,11):
        link = soup.select('div.views-row-%s > div.node > h2 > a'%i)[0]['href']
        article_url = DOMAIN + link
        article_urls.append(article_url)
# print(article_urls)
movie_dict_list = []
# for art in article_urls[:10]:
for art in article_urls:
    resp = requests.get(art, headers = headers)
    soup = BeautifulSoup(resp.text, 'lxml')
    
    title = soup.select('div.breadcrumb > a.active')[0].text
    a = soup.select('div.content.clear-block > p > span')
    
    if '《' in title:  
        movie_dict['chname']  = soup.select('div.breadcrumb > a.active')[0].text.split('》')[0].split('《')[1]   
        movie_dict['comment'] = ["".join([span.text for span in a])]
                  
        movie_dict_list.append({
                                        'chname'   : movie_dict['chname'],
                                        'comment'  : movie_dict['comment'] 
                })
#         counter+=1
#         print(counter)
# movie_dict_list

In [6]:
with open('C:\\Users\\Java\\ETL\\movie\\Files_for_git\\hkfilmcritics.json', 'w', encoding = 'UTF-8') as f:
    f.write(json.dumps(movie_dict_list, ensure_ascii = False))

In [7]:
len(movie_dict_list)
# len(InJ_compare)

865

In [8]:
upload_list = []

for m in movie_dict_list:
    for i in InJ_compare:
        if m['chname'] == i[3]:
            m['imdb_id'] = i[0]
            upload_list.append(m)
            break
upload_list

[{'chname': '萬世魔星',
  'comment': [' 耶穌基督和祂的時代一向是電影人喜歡的題材，由默片年代開始，祂和祂身邊的門徒已經不斷登上大銀幕，加上虛構的同時代人物（例如賓虛），「公元」頭幾十年的世界變成了史詩電影源源不絕的靈感泉源。在這些電影出場的人物之中，最具爭議性的可能還是英國喜劇組合踎低噴飯（Monty Python）作品《萬世魔星》（Life of Brian）的主人翁「拿撒勒的拜恩」。這部1979年的電影歷年招來的抗議和禁映不少，新舊教教士以至猶太拉比中都不乏這部電影的敵人指控其褻瀆。雖然，有些指控者其實遲了入場，連拜恩不是耶穌這點都沒有搞清楚……但說到底，這部喜劇和那些滿足千千萬萬基督徒想像的「聖經史詩片」有些什麼異同，搞到三十多年後還不時傳出禁映的消息？作為一個喜劇組合，踎低噴飯一向會拿西方世界的文化傳統做文章，在《萬世魔星》之前，這類作品最成功的例子是《踎低噴飯：勇奪聖杯》（Monty Python and the Holy Grail，1975）。這部把大英帝國的阿瑟王與圓桌武士傳說徹底反轉的電影十分成功，於是他們下一部電影把目光轉向更為久遠宏大的西方傳統──聖經故事，但和《勇奪聖杯》把阿瑟王拍成小丑般的傻瓜不同，《萬世魔星》裡的確有耶穌出現，不過他的出現是完全正常的，可說是合乎福音書中的形象。耶穌出現在電影片頭字幕結束後，主角拜恩已由初生嬰兒變成了男子漢，他和他的母親是在山上聆聽耶穌「登山寶訓」的群眾之一。本片處理這個著名的場面和其他聖經電影最大的不同，在於鏡頭很快就從發表演說的耶穌身邊拉開，放在邊緣位置聽他講話的人群上。這種取態其實是一個很日常的思考，在古代世界沒有擴音機，發表演說的人是如何讓聽眾知道自己所講？或者說你站立的位置在人群邊緣的話，又要怎樣接收「山上」講者的說話？其實可以是很抽離的一件事，所以拜恩的媽媽不耐煩要去參加石刑，也有人把「peacemaker」聽成了「cheesemaker」，笑料就是從這些地方來，耶穌本人還是很嚴肅正經的（主角拜恩其實也是一個嚴肅正經的人）。踎低噴飯經營的喜劇場面，說起來是對古代社會「日常生活」細節的發揮，像石刑那場戲，當時的猶太人女性不可以參與公眾集會，那她們如果想要參加這種集會，要做些甚麼事，會發生甚麼情況呢？又例如，面對當時遍地的真假先知，猶太人又是如何去挑選、分辨的呢？是他

In [5]:
len(upload_list)

143

In [6]:
with open('C:\\Users\\Java\\ETL\\movie\\Files_for_git\\hkfilmcritics_final.json', 'w', encoding = 'UTF-8') as f:
    f.write(json.dumps(upload_list, ensure_ascii = False))

In [9]:
#upload_list的pickle
with open("C:\\Users\\Java\\ETL\\movie\\HKfilm_upload_list","wb") as f:
    pickle.dump(upload_list,f) 